In [ ]:
import numpy as np
import scipy.linalg as linalg
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as linalg
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import ion
from math import *
import sys

def gasoilf( t, y, a ):
    """ f fonction dÃ©finissant le systÃ¨me d'EDO
    t: temps
    y: solution
    a: paramÃ¨tre 
    yprime: valeur de la fonction
    """
    return np.array([-(a[0]+a[2])*(y[0]**2), a[0]*(y[0]**2) - a[1]*y[1]])
    
def gasoildyf( t, y, a ):
    
    dyf = np.array([[-2*y[0]*(a[0]+a[2]),2*a[0]*y[0]],[0,-a[1]]])
    
    return dyf
    """ gasoildyf Jacobien de gasoilf par rapport Ã  y
       t: temps
       y: solution
       a: paramÃ¨tre 
       dyf: jacobienne / y
    """
    # A faire par vous mÃªme
    # Cf texte projet

def gasoildaf( t, y, a ):
    
    daf = np.array([[-y[0]**2, y[0]**2],[0, -y[1]],[-y[0]**2, 0]])
    return daf
    """ gasoildyf Jacobien de gasoilf par rapport Ã  y
       t: temps
       y: solution
       a: paramÃ¨tre 
       daf: jacobienne / a
    """
    #yhu
    # Cf texte projet

def direct(f,t0, tf, h, a, y):
    """direct  integration EDO 
     
    f: fonction dÃ©finissant l'EDO
    prototype yprim = f(t,y,a)
    t0, tf: intervalle d'intÃ©gration
    h:  pas de discrÃ©tisation
    a: paramÃ¨tres du systÃ¨me
    y: condition initiale

    etat: solution sur tout l'intervalle 
    taille: Nt x length(y) 
    """
    Nt = int((tf-t0)/h)
    etat=[y]
    for n in range(Nt):
        y = y + h*f(h*n,y,a)
    etat = np.vstack((etat, y))
    print(etat)
    return etat

def linearise( jyf, t0, tf, h, a, dy, y, b ):
    """linearise IntÃ©gration du systÃ¨me linÃ©arisÃ© de l'EDO

    jyf: jacobienne en y de la fonction dÃ©finissant l'EDO
    prototype yprim = jyf(t,y,a)
    t0, tf: intervalle d'intÃ©gration
    h:  pas de discrÃ©tisation
    a: paramÃ¨tres du systÃ¨me
    y: etat direct, taille (nbre equations) x (nbre pas de temps)
    b: terme source (mÃªme taille que y)
    
    detat: solution sur tout l'intervalle 
    taille: mÃªme taille que y 
    """
    Nt = int((tf-t0)/h)
    detat=[0]
    for k in range(Nt):
        dy = dy + h*(jyf(h*k,y[:,k],a)+b[:,k])
    detat = np.vstack((detat,dy)) 
    return detat

def adjoint(jyf,  t0, tf, h, a, y, r):
    """adjoint  IntÃ©gration du systÃ¨me adjoint

    jyf: jacobienne en y de la fonction dÃ©finissant l'EDO
    prototype yprim = jyf(t,y,a)
    t0, tf: intervalle d'intÃ©gration
    h:  pas de discrÃ©tisation
    a: paramÃ¨tres du systÃ¨me
    y: etat direct, taille (nbre equations) x (nbre pas de temps)
    r: terme source (mÃªme taille que y)
    
    etatdj: solution sur tout l'intervalle 
    taille: mÃªme taille que y 
    """
    # A faire par vous mÃªme
    # Cf Direct
    # !! Temps rÃ©trograde
    etatdj=[0]
    Nt = int((tf-t0)/h)
    for k in range(Nt):
        etatdj = etatdj+h*(-r[k]+np.transpose(jyf(h*k,y[k],a)))
        etatdj = np.vstack((etatdj,y))
    return etatdj
        
def tstadj(f, jyf, t0, tf, h, y0, aref ):
    """tstadj Test du produit scalaire pour valider l'Ã©tat adjoint

    f, jyf fonction dÃ©finissant l'EDO et sa jacobienne
    t0, tf, h, y0: pour l'Ã©quation diffÃ©rentielle (cf direct)
    aref: paramÃ¨tre oÃ¹ on calcule l'adjoint
    
    bp = produit scalaire (b, p)
    dy2 = produit scalaire (dy, dy)
    avec b alÃ©atoire, 
    dy = solution Ã©tat direct linÃ©arisÃ© (2nd membre b)
    p = solution Ã©tat adjoint (2nd membre dy)
    """

    yref= direct(f, t0, tf, h, aref, y0)
    dy0 = np.zeros(yref[0,:].size)    
    b = np.random.random_sample(yref.shape)
    dy = linearise(jyf,t0,tf,h,aref,dy0,yref,b)
    p = adjoint(jyf,t0,tf,h,aref,yref,dy)
    bp = np.vdot(p,b)
    dy2=np.vdot(dy,dy)
    return bp, dy2
    
def simul( a, f, jyf, jaf, t0, tf, h, y0, tobs, dobs ):
    """ simul Fonction coÃ»t pour les moindres ccarÃ©s
    
    f, jyf, jaf, fonctions dÃ©finissant l'EDO
    t0, tf, h, y0: paramÃ¨tres pour l'EDO (cf direct)
    a: paramÃ¨tre oÃ¹ on calcule le gradient
    tobs, dobs: instant et valeurs des observations
    
    cout: valeur de la fonction 
    """

    # Cette fonction doit calculer cout = 1/2 (y(tobs, a) - dobs)^2
    # oÃ¹ y(a) est le rÃ©sultat de l'Ã©tat direct
    
    # A faire par vous mÃªme

   

def gradient( a, f, jyf, jaf, t0, tf, h, y0, tobs, dobs ):
    """gradient  Calcul du gradient de la fonctionnelle dÃ©finie dans simul
    
    % f, jyf, jaf, fonctions dÃ©finissant l'EDO
    % t0, tf, h, y0: paramÃ¨tres pour l'EDO (cf direct)
    % a: paramÃ¨tre oÃ¹ on calcule le gradient
    % tobs, dobs: instant et valeurs des observations
    
    % gradient
    """

    y = direct(f, t0, tf, h, a, y0);
    r=np.zeros(y.shape);
    nobs=np.array(np.floor(tobs/h), dtype=np.int64)

    # A faire par vous mÃªme
    
    return g
        

def tstgrad( f, jyf, jaf, t0, tf, h, y0, a0, delta, tobs, dobs ):
    """ tstgrad validation du gradient par diffÃ©rences finies
    
     f, jyf, jaf, fonctions dÃ©finissant l'EDO
     t0, tf, h, y0: paramÃ¨tres pour l'EDO (cf direct)
     a0: paramÃ¨tre oÃ¹ on calcule le gradient
     (vecteur) des pas utilisÃ©s pour les diffÃ©rences finies
     tobs, dobs: instant et valeurs des observations
    
     gref: dÃ©rivÃ©e directionnelle Ã©tat adjoint (vecteur (1, 1,...,1))
     g = calculs par DF
    """
    
    cref=simul(a0, f, jyf, jaf, t0, tf, h, y0, tobs, dobs);
    gref=gradient(a0, f, jyf, jaf, t0, tf, h, y0, tobs, dobs);
    gref=np.sum(np.sum(gref));
    
    c=[];
    for i in np.arange(delta.size):
        c=np.hstack((c, simul(a0+delta[i], f, jyf, jaf, t0, tf, h, y0, tobs, dobs)));
        
    g=(c-cref)/delta;
    return gref, g


def main():

    #t0, tf, h, y0 paramÃ¨tres pour l'Ã©qaution diffÃ©rentielle (cf direct)
    #tobs: instants d'observation
    #Neq, Nparams: nbre d'Ã©quations, de paramÃ¨tres
    #datafile: fichier contenant les observations
    #f, dyf, daf : fonctions dÃ©finissant le systÃ¨me et ses dÃ©rivÃ©es

    t0=0
    tf=1
    h=0.0002
    y0=[1, 0]
    tobs= np.arange(0,21)*0.05
    Neqs=2; Nparams=3;
    datafile='projet2.dat'
    f=gasoilf
    jyf=gasoildyf
    jaf=gasoildaf
    nobs=np.array(np.floor(tobs/h), dtype=np.int64)
    t= np.arange(t0, tf+h, h)
    aref=np.ones(Nparams)
    yref= direct(f, t0, tf, h, aref, y0)
    """plt.figure()
    plt.subplot(211)
    plt.plot(t,yref)
    plt.xlabel('t')
    plt.ylabel('y')"""

    bp, dy2 = tstadj(f, jyf, t0, tf, h, y0, aref)
    print('Test du produit scalaire : bp={0:18.15e}, dy2={1:18.15e}'.format(bp, dy2))
    
    """dobs = np.random.rand(tobs.size, Neqs);
    delta=np.logspace(-2,-11,10) ;  
    [gref, g]=tstgrad(f, jyf, jaf, t0, tf, h, y0, aref, delta, tobs, dobs);
    print(gref)
    print(g)
    print((g-gref)/g)

    dobs = yref[nobs,:]
    ainit = aref*1.2;
    resopt = minimize(simul, ainit, method='BFGS', jac=gradient, args=(f, jyf, jaf, t0, tf, h, y0, tobs, dobs), options={'disp': True})
    aopt=resopt.x
    print(aopt)
    ysim=direct(f, t0, tf, h, aopt, y0)
    plt.subplot(212)
    plt.plot(t,ysim);
    plt.xlabel('t'); plt.ylabel('y')
    
    dobs=np.loadtxt('projet2.dat')
    ainit=np.random.rand(Nparams)
    print('a initial {}'.format(ainit))
    yinit= direct(f,t0,tf,h,ainit,y0);
    resopt = minimize(simul, ainit, method='BFGS', jac=gradient, args=(f, jyf, jaf, t0, tf, h, y0, tobs, dobs), options={'disp': True})
    aopt=resopt.x
    print('a optimal {}'.format(aopt))
    yopt= direct(f,t0,tf,h,aopt,y0);
    plt.figure()
    plt.plot(t, yinit, label='param. initial')
    plt.plot(t,yopt, label='param optimal')
    plt.plot(tobs, dobs, '+', label='Observations')
    plt.xlabel('t'); plt.ylabel('y'); plt.legend()"""

if __name__ == "__main__":
    # execute only if run as a script
    main()
    plt.show()